In [3]:
# Basic Imports
import numpy as np

In [2]:
# Reference: https://ieeexplore.ieee.org/document/5980409
# Convention: WRB = WRC * CRB (means body is first rotated via pitch and roll, then rotated via yaw into world frame)

# Angular velocity of body frame in world frame where p q r are the angular velocities in the body frame and xb yb zb are the body frame axes (3D unit vectors).
# p, q, r is roll, pitch, yaw respectively.
def body_angular_velocity_to_world(p, q, r, xb, yb, zb):
    return np.array([
        p * xb + q * yb + r * zb,
        p * yb + q * zb - r * xb,
        p * zb - q * xb + r * yb
    ]).T



In [1]:
# Each rotor has an angular speed wi and produces a force Fi and a moment Mi.
# Force is in the direction of the rotor axis and is perpendicular to the plane of rotation, while moment is in the plane of rotation.

# The force Fi is given by:
# Fi = Fk * wi^2
# where Fk is a constant.

# The moment Mi is given by:
# Mi = Mk * wi^2
# where Mk is a constant.

def rotor_force(wi, Fk):
    return Fk * wi**2
def rotor_moment(wi, Mk):
    return Mk * wi**2

# Given motor dynamics are comparatively fast, we can assume they are instantaneous. This means we can turn the control inputs into forces and moments.
# u1 is the net body force, u2 is the roll moment, u3 is the pitch moment, and u4 is the yaw moment.
# L is the distance from the center of rotation to the vehicle center.

# The net body force is given by:
    # u1 = F1 + F2 + F3 + F4
# The roll moment is given by:
    # u2 = F2*L - F4*L = L*(F2 - F4)
# The pitch moment is given by:
    # u3 = F3*L - F1*L = L*(F3 - F1)
# The yaw moment is given by:
    # u4 = M1 - M2 + M3 - M4

# Therefore, as a matrix, separating out the angular velocities, we can write:
def rotor_forces_moments(w1, w2, w3, w4, L, Fk, Mk):
    motors = np.array([
        [Fk, Fk, Fk, Fk],
        [0, Fk*L, 0, -Fk*L],
        [-Fk*L, 0, Fk*L, 0],
        [Mk, -Mk, Mk, -Mk]
    ])
    speeds = np.array([w1**2, 
                       w2**2, 
                       w3**2, 
                       w4**2])
    forces_moments = motors @ speeds
    return forces_moments

In [ ]:
# The position of the quadrotor in world frame is given by "r".
# The acceleration treating the quad as a point mass in world frame is given by:
# m * rddot = -mg + u1
# where mg is in world frame and u1 is the net body force in the Z direction of the quadrotor in world frame.
# You can imagine a tilted plane as the quadrotor with the z axis pointing up. After applying the rotation matrix, it becomes a 3d vector in world frame.

def quad_acceleration(u1 : np.ndarray, m, g):
    # assumes u1 is in world frame already
    accel = -m * g + u1
    return accel / m

In [ ]:
# Angular acceleration is a bit more complicated.
# It follows the standard rigid body dynamics equations
#   I * omega_dot = -omega x (I * omega) + M
# where I is the inertia matrix, omega is the angular velocity, and M is the moment.
# NOTE: Luckily, this controller gets away without using the inertial matrix
#   since none of the equations depend on angular acceleration.
# The inertia matrix is given by:
#   I = diag(Ixx, Iyy, Izz)
# The angular velocity is given by:
#   omega = [p, q, r]
# The moment is given by:
#   M = [u2, u3, u4]

# Therefore, the angular acceleration is given by:
#   omega_dot = I^-1 * (-omega x (I * omega) + M)
# where I^-1 is the inverse of the inertia matrix.
# The angular velocity is given by:
#   omega = [p, q, r]
# The moment is given by:
#   M = [u2, u3, u4]
# The inertia matrix is given by:
#   I = diag(Ixx, Iyy, Izz)

In [ ]:
# Therefore we can write a state matrix for the quadrotor as:
# phi, theta, psi describe how to rotate the quadrotor from world to body frame (in essence, the orientation of the quadrotor in world frame).
# p, q, r describe the angular velocity of the quadrotor in body frame.
# everything else is in world frame.
# x = [x, y, z, phi, theta, psi, x_dot, y_dot, z_dot, p, q, r].T

In [ ]:
# Quadrotors are differentially flat (just take this for granted pain in the butt to prove).
# Means that the states and the inputs can be written as algebraic functions of four carefully selected flat outputs and their derivatives.
# We can define the flat outputs as:
#   x = [x, y, z, psi(yaw)]

In [ ]:
# OK lets start with the control now. The paper uses a PD + feedforward controller.
# Given a trajectory defined by the flat outputs where sigma is the flat outputs = [r(t).T, psi(t)]

# eP = r - r(t)
# eV = rdot - rdot(t)

# Essentially ep is position error and ev is velocity error.
# Both are in world frame.

In [ ]:
# From this we can then determine the desired force vector first in world frame.
# mg is multiplied by the z axis in world frame (unit vector also called E3 [0,0,1]) so the force is in a vector in world frame.
# Fdes = -Kp * eP - Kv * eV + m*g*Zw + m*rddot(t)

# Kp and Kv are positive definite matrices, they are the proportional and derivative gains respectively. Also Fdes must not be 0.

#NOTE: What is a positive definite matrix?
# A positive definite matrix is a square matrix that has all positive eignevalues.
# This means that matrix M for any non-zero vector x, the following holds:
#   x^T * M * x > 0 for all x != 0
# Since we want to decouple each axis, we can use a diagonal matrix for Kp and Kv and set the diagonal elements to positive values (which are the gains for each axis).
# If the axis's were coupled, we would have to use a full matrix and ensure it is positive definite by checking the eigenvalues.

In [ ]:
# Since Fdes is in world frame, we need to determine the component of it that is in the body z-frame.

# u1 = Fdes dot Zb

# This returns a scalar for the desired force in the body z-frame.

In [ ]:
# Next we need to determine the next 3 inputs u2, u3, u4.
# While U1 is based on the desired force and the component thats already in the right direction,
# the next step is to rotate the quadrotor so so that Zb is aligned with the desired Force vector (Fdes).

In [ ]:
# Given that we know the desired yaw angle from the trajectory, we can build the desired rotation matrix from the world frame to the body frame.

# First we go to the intermediate frame using psi(traj yaw) and align the x-axis so we have the correct frame reference for the pitch and roll.
#   Xc, des = [cos(psi), sin(psi), 0].T
# What this does is it aligns the x-axis of the quadrotor with the desired yaw angle in world frame.
# We apply yaw first because we are going from World to Body frame (BRW).

# To get the unit vector for the desired body z-axis we can simply take the desired force vector and normalize it.
# #   Zb_des = Fdes / ||Fdes||
# This is in world frame

# Then we can take the cross product of Zb and Xc which gives us the orthogonal vector to both Zb and Xc that is Yb.
# Note that dividing a vector by its norm/magnitude gives us the unit vector in that direction.
# Note that these are all desired (ie Yb_des, Zb_des, Xc_des) and not the actual body frame vectors.
#   Yb = Zb x Xc / ||Zb x Xc||

# Now that we have Xb and Yb, we can solve for Zb which is the cross product of Xb and Yb.
# They are all unit vectors, so we can use the cross product without normalizing them.
#   Xb = Yb x Zb

# NOTE: A singularity occurs when Zb is parallel to Xc, which means that the quadrotor is in a vertical position and cannot roll or pitch, but also leads to numerical instability.
# So when Xc cross Zb is < epsilon, we recompute the frame using a -xB and -yB instead of xB and yB and then see which is closer to the actual current frame.
# To check which is closer, we can compute the dot product of the current frame with the desired frame and see which is closer to 1.

In [ ]:
# The actual rotation matrix using Xb, Yb, Zb is given by:
# R_des = [Xb, Yb, Zb].T
def rotation_matrix_from_body_frame(Xb, Yb, Zb):
    return np.array([
        [Xb[0], Yb[0], Zb[0]],
        [Xb[1], Yb[1], Zb[1]],
        [Xb[2], Yb[2], Zb[2]]
    ])

In [ ]:
# Now that we actually have the rotation matrix, we can compute the desired roll and pitch angles.

# First the orientation error is given by:
#   Er = 1/2 * (R_des.T * R_current - R_current.T * R_des)V
# This provides a skew-symmetric matrix that represents the orientation error.
# The V at the end is a vee mapping operator that converts a skew-symmetric matrix into a vector of the form [x, y, z].

# NOTE: A matrix is skew-symmetric if its transpose is equal to its negative (A = -A^T). For a 3x3 matrix, 
# this means that the elements on the diagonal are zero, and the elements above the diagonal are the negative 
# of the corresponding elements below the diagonal. 

In [ ]:
# The angular velocity error is given by:
#   eOmega = omega - omega_des
# where omega is the current angular velocity and omega_des is the desired angular velocity.

In [ ]:
# NOTE: But how do you get the desired angular velocity?
# Since all of our math is in world frame: Zb = Wrb @ [0,0,1].T (converting the unit vector in the z direction of the body frame to world frame).

# We can start by solving the derivative of the net force vector of the quadrotor based on newtonian mechanics.
# Originally, we have:
    # m * a = -mg(Zw) + u1(Zb) # where Zw is the unit vector in the z direction of the world frame and Zb is the unit vector in the z direction of the body frame.
    # To put Zb into world frame we would use the rotation matrix R from body to world frame.
# Its derivative is:
    # m * a_dot = u1_dot(Zb) + omega x u1(Zb)


# We can now do some quirky math with this to change its form. If we take the dot product of the above equation with Zb, we can isolate u1_dot.
    # Zb dot m*a_dot = Zb dot (u1_dot(Zb) + omega x u1(Zb))
    # Zb dot m*a_dot = 1*U1dot + Zb dot (omega x u1(Zb)) # dot product with itself is 1
    # Zb dot m*a_dot = u1_dot + 0 # dot and then cross product with itself is 0

# Therefore this gives you the rate of change of thrust purely from derivatives of flat outputs.
#   u1_dot  = Zb dot m*a_dot

# So now if you replace u1_dot in the original equation, we can write it as:
# m*a_dot = (Zb dot m*a_dot) * Zb + omega x u1(Zb)
# This eventually then turns into the next equation Hw below.

# We can define Hw as the projection of (m / u1) * a_dot onto the Xb - Yb plane and write it in terms of p and q.
#   Hw = omega x Zb = (m / u1) * (a_dot - (Zb dot a_dot)Zb)

# NOTE: Why is this important?
# omega x Zb will always return a vector that is perpendicular to Zb, which means it will always be in the Xb - Yb plane.
# Therefore this literally removes the a_dot component in the Zb direction and leaves us with the component perpendicular to Zb, 
# which is in the Xb - Yb plane (meaning pitch and roll).

# Since Hw has both the p and q components, we can grab them by taking the dot product with Yb and Xb respectively.
# Specifically, the dot product measures the magnitude of the component of one vector that lies along the direction of another vector. 
# This allows us to then write it as:
#   p = -Hw dot Yb
#   q = Hw dot Xb
#   r = psi_dot(Zw) dot Zb

# RECALL: omega  = p * Xb + q * Yb + r * Zb

In [ ]:
# Now that we have both the orientation and angular velocity errors, we can compute U2, U3, and U4:

#   [u2, u3, u4] = -Kr * Er - Kw * eOmega

# Where Kr and Kw are diagonal gain matrices for the orientation and angular velocity errors respectively.

In [ ]:
# To get the actual rotor speeds to achieve the inputs, we invert the linearization of the rotor forces and moments:
# w^2 = M^-1 * [u1, u2, u3, u4].T
# Then clamp so that the speeds are non-negative and take the square root.
# if w^2 > 0 then w = sqrt(w^2) else w = 0.

# And bam, now you have the rotor speeds to achieve the desired trajectory.